### Semana 3 - Churn Prediction


##### Predicción clientes que quieren cancelar servicios de una compañia
#### DictVectorizer, Logistic Regression

## 3.2 Preparación de datos

In [100]:
import pandas as pd 
import numpy as np

In [101]:
import matplotlib.pyplot as plt

In [102]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/refs/heads/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'

In [103]:
!wget $data -O data-week-3.csv #el ! es para ejecutar una linea de comando de shell

--2025-06-27 20:00:59--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/refs/heads/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 977501 (955K) [text/plain]
Saving to: ‘data-week-3.csv’

data-week-3.csv     100%[===================>] 954.59K  --.-KB/s    in 0.02s   

2025-06-27 20:00:59 (39.0 MB/s) - ‘data-week-3.csv’ saved [977501/977501]



In [104]:
df = pd.read_csv('data-week-3.csv')
df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


In [105]:
df.head().T #permite leer todas las columnas de los primeros clientes

,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No


### Estandarizamos datos

In [106]:
df.columns = df.columns.str.lower().str.replace(' ','_') #cambiamos el nombre de columnas a uno estandarizado

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index) #retorna el nombre de las columnas categoricas

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ','_') #estandarizamos los datos de cada columna

In [107]:
df

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,7590-vhveg,female,0,yes,no,1,no,no_phone_service,dsl,no,...,no,no,no,no,month-to-month,yes,electronic_check,29.85,29.85,no
1,5575-gnvde,male,0,no,no,34,yes,no,dsl,yes,...,yes,no,no,no,one_year,no,mailed_check,56.95,1889.5,no
2,3668-qpybk,male,0,no,no,2,yes,no,dsl,yes,...,no,no,no,no,month-to-month,yes,mailed_check,53.85,108.15,yes
3,7795-cfocw,male,0,no,no,45,no,no_phone_service,dsl,yes,...,yes,yes,no,no,one_year,no,bank_transfer_(automatic),42.30,1840.75,no
4,9237-hqitu,female,0,no,no,2,yes,no,fiber_optic,no,...,no,no,no,no,month-to-month,yes,electronic_check,70.70,151.65,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-resvb,male,0,yes,yes,24,yes,yes,dsl,yes,...,yes,yes,yes,yes,one_year,yes,mailed_check,84.80,1990.5,no
7039,2234-xaduh,female,0,yes,yes,72,yes,yes,fiber_optic,no,...,yes,no,yes,yes,one_year,yes,credit_card_(automatic),103.20,7362.9,no
7040,4801-jzazl,female,0,yes,yes,11,no,no_phone_service,dsl,yes,...,no,no,no,no,month-to-month,yes,electronic_check,29.60,346.45,no
7041,8361-ltmkd,male,1,yes,no,4,yes,yes,fiber_optic,no,...,no,no,no,no,month-to-month,yes,mailed_check,74.40,306.6,yes


#### Revisamos los tipos de datos que tenemos

In [108]:
df.dtypes

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges         object
churn                object
dtype: object

In [109]:
df.iloc[488]

customerid                         4472-lvygi
gender                                 female
seniorcitizen                               0
partner                                   yes
dependents                                yes
tenure                                      0
phoneservice                               no
multiplelines                no_phone_service
internetservice                           dsl
onlinesecurity                            yes
onlinebackup                               no
deviceprotection                          yes
techsupport                               yes
streamingtv                               yes
streamingmovies                            no
contract                             two_year
paperlessbilling                          yes
paymentmethod       bank_transfer_(automatic)
monthlycharges                          52.55
totalcharges                                _
churn                                      no
Name: 488, dtype: object

In [110]:
tc = pd.to_numeric(df.totalcharges, errors = 'coerce') #ignora los datos faltas en el caso de totalcharges

In [111]:
df[tc.isnull()][['customerid','totalcharges']] #muestra los datos faltantes

,customerid,totalcharges
488,4472-lvygi,_
753,3115-czmzd,_
936,5709-lvoeq,_
1082,4367-nuyao,_
1340,1371-dwpaz,_
3331,7644-omvmy,_
3826,3213-vvolg,_
4380,2520-sgtta,_
5218,2923-arzlg,_
6670,4075-wkniu,_


In [112]:
df.totalcharges = pd.to_numeric(df.totalcharges, errors = 'coerce') #convertimos los valores en el dataframe
df.totalcharges = df.totalcharges.fillna(0) #replazamos los na 

### Convertir la columna Churn en una binaria

In [113]:
df.churn = (df.churn == 'yes').astype(int) #si es True, convierte en 1, de lo contrario, en 0

In [114]:
df

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,7590-vhveg,female,0,yes,no,1,no,no_phone_service,dsl,no,...,no,no,no,no,month-to-month,yes,electronic_check,29.85,29.85,0
1,5575-gnvde,male,0,no,no,34,yes,no,dsl,yes,...,yes,no,no,no,one_year,no,mailed_check,56.95,1889.50,0
2,3668-qpybk,male,0,no,no,2,yes,no,dsl,yes,...,no,no,no,no,month-to-month,yes,mailed_check,53.85,108.15,1
3,7795-cfocw,male,0,no,no,45,no,no_phone_service,dsl,yes,...,yes,yes,no,no,one_year,no,bank_transfer_(automatic),42.30,1840.75,0
4,9237-hqitu,female,0,no,no,2,yes,no,fiber_optic,no,...,no,no,no,no,month-to-month,yes,electronic_check,70.70,151.65,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-resvb,male,0,yes,yes,24,yes,yes,dsl,yes,...,yes,yes,yes,yes,one_year,yes,mailed_check,84.80,1990.50,0
7039,2234-xaduh,female,0,yes,yes,72,yes,yes,fiber_optic,no,...,yes,no,yes,yes,one_year,yes,credit_card_(automatic),103.20,7362.90,0
7040,4801-jzazl,female,0,yes,yes,11,no,no_phone_service,dsl,yes,...,no,no,no,no,month-to-month,yes,electronic_check,29.60,346.45,0
7041,8361-ltmkd,male,1,yes,no,4,yes,yes,fiber_optic,no,...,no,no,no,no,month-to-month,yes,mailed_check,74.40,306.60,1


## 3.3 Dividir el data frame en 3, validation, train y test con scikit-learn

In [115]:
from sklearn.model_selection import train_test_split #divide el data frame de manera aleatoria

In [116]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1) #se pasa el porcentaje y semilla de random

In [117]:
len(df_full_train), len(df_test) #tamaños de los dataframes

(5634, 1409)

In [118]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [119]:
len(df_test),len(df_train),len(df_val)

(1409, 4225, 1409)

#### Reseteamos los índices

In [120]:
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

#### Creamos los vectores y, de val, train, validation

In [121]:
y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values

#### Eliminamos la variable churn de los dataframes divididos

In [122]:
del df_train['churn']
del df_val['churn']
del df_test['churn']

In [123]:
df_train

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges
0,8015-ihcgw,female,0,yes,yes,72,yes,yes,fiber_optic,yes,yes,yes,yes,yes,yes,two_year,yes,electronic_check,115.50,8425.15
1,1960-uycnn,male,0,no,no,10,yes,yes,fiber_optic,no,yes,yes,no,no,yes,month-to-month,yes,electronic_check,95.25,1021.55
2,9250-wypll,female,0,no,no,5,yes,yes,fiber_optic,no,no,no,no,no,no,month-to-month,no,electronic_check,75.55,413.65
3,6786-obwqr,female,0,yes,yes,5,yes,no,fiber_optic,no,no,no,no,yes,no,month-to-month,yes,electronic_check,80.85,356.10
4,1328-euzhc,female,0,yes,no,18,yes,no,no,no_internet_service,no_internet_service,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,20.10,370.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4220,1309-xgfsn,male,1,yes,yes,52,yes,yes,dsl,no,yes,yes,no,yes,yes,one_year,yes,electronic_check,80.85,4079.55
4221,4819-hjpiw,male,0,no,no,18,no,no_phone_service,dsl,no,no,no,no,no,no,month-to-month,no,mailed_check,25.15,476.80
4222,3703-vavcl,male,0,yes,yes,2,yes,no,fiber_optic,no,no,yes,yes,no,yes,month-to-month,no,credit_card_(automatic),90.00,190.05
4223,3812-lrzir,female,0,yes,yes,27,yes,yes,no,no_internet_service,no_internet_service,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,electronic_check,24.50,761.95


## 3.4 EDA

In [124]:
df_full_train = df_full_train.reset_index(drop=True)

#### Revisamos datos faltantes

In [125]:
df_full_train.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [126]:
df_full_train.churn.value_counts(normalize=True) #cuenta el número que hay de cada valor, normalize muestra el porcentaje

churn
0    0.730032
1    0.269968
Name: proportion, dtype: float64

In [127]:
global_churn_rate = df_full_train.churn.mean()
global_churn_rate

np.float64(0.26996805111821087)

#### Miramos los tipos de variables

In [128]:
df_full_train.dtypes

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges        float64
churn                 int64
dtype: object

In [129]:
numerical = ['tenure','monthlycharges','totalcharges']
numerical

['tenure', 'monthlycharges', 'totalcharges']

In [130]:
df_full_train.columns

Index(['customerid', 'gender', 'seniorcitizen', 'partner', 'dependents',
       'tenure', 'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod', 'monthlycharges', 'totalcharges', 'churn'],
      dtype='object')

In [131]:
categorical = [col for col in df_full_train.columns if col not in numerical + ['customerid', 'churn']]
categorical

['gender',
 'seniorcitizen',
 'partner',
 'dependents',
 'phoneservice',
 'multiplelines',
 'internetservice',
 'onlinesecurity',
 'onlinebackup',
 'deviceprotection',
 'techsupport',
 'streamingtv',
 'streamingmovies',
 'contract',
 'paperlessbilling',
 'paymentmethod']

In [132]:
df_full_train[categorical].nunique() #retorna el numero de valores únicos de cada columna

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

## 3.5 Importancia de las características

In [133]:
churn_female = df_full_train[df_full_train.gender == 'female'].churn.mean() #porcentaje de mujeres que se van
churn_female

np.float64(0.27682403433476394)

In [134]:
churn_male = df_full_train[df_full_train.gender == 'male'].churn.mean() #porcentaje de hombres que se van
churn_male

np.float64(0.2632135306553911)

In [135]:
global_churn = df_full_train.churn.mean()
global_churn

np.float64(0.26996805111821087)

In [136]:
churn_partner = df_full_train[df_full_train.partner == 'yes'].churn.mean() #porcentaje de personas con pareja que se van
churn_partner

np.float64(0.20503330866025166)

In [137]:
churn_no_partner = df_full_train[df_full_train.partner == 'no'].churn.mean() #porcentaje de personas sin pareja que se van
churn_no_partner

np.float64(0.3298090040927694)

#### Risk ratio

In [138]:
churn_partner / global_churn

np.float64(0.7594724924338315)

In [139]:
churn_no_partner / global_churn

np.float64(1.2216593879412643)

In [140]:
from IPython.display import display

In [141]:
for c in categorical: #mostrar el riesgo y diferencia de cada variable
    print(c)
    df_group = df_full_train.groupby(c).churn.agg(['mean','count'])
    df_group['diff'] = df_group['mean'] - global_churn #agregamos al dataframe la diferencia y el riesgo
    df_group['risk'] = df_group['mean'] / global_churn
    display(df_group)
    print()
    print()

gender


,mean,count,diff,risk
gender,,,,
female,0.276824,2796,0.006856,1.025396
male,0.263214,2838,-0.006755,0.974980




seniorcitizen


,mean,count,diff,risk
seniorcitizen,,,,
0,0.242270,4722,-0.027698,0.897403
1,0.413377,912,0.143409,1.531208




partner


...

,mean,count,diff,risk
contract,,,,
month-to-month,0.431701,3104,0.161733,1.599082
one_year,0.120573,1186,-0.149395,0.446621
two_year,0.028274,1344,-0.241694,0.104730




paperlessbilling


,mean,count,diff,risk
paperlessbilling,,,,
no,0.172071,2313,-0.097897,0.637375
yes,0.338151,3321,0.068183,1.252560




paymentmethod


,mean,count,diff,risk
paymentmethod,,,,
bank_transfer_(automatic),0.168171,1219,-0.101797,0.622928
credit_card_(automatic),0.164339,1217,-0.105630,0.608733
electronic_check,0.455890,1893,0.185922,1.688682
mailed_check,0.193870,1305,-0.076098,0.718121


## 3.6 Mutual Information

In [142]:
from sklearn.metrics import mutual_info_score

In [143]:
mutual_info_score(df_full_train.churn, df_full_train.contract)

np.float64(0.0983203874041556)

In [144]:
mutual_info_score(df_full_train.churn, df_full_train.gender)

np.float64(0.0001174846211139946)

#### Los valores más altos representan más correlación

In [145]:
def mutual_info_churn_score(series):
    return mutual_info_score(series, df_full_train.churn)

In [146]:
mi = df_full_train[categorical].apply(mutual_info_churn_score)
mi.sort_values(ascending=False)

contract            0.098320
onlinesecurity      0.063085
techsupport         0.061032
internetservice     0.055868
onlinebackup        0.046923
deviceprotection    0.043453
paymentmethod       0.043210
streamingtv         0.031853
streamingmovies     0.031581
paperlessbilling    0.017589
dependents          0.012346
partner             0.009968
seniorcitizen       0.009410
multiplelines       0.000857
phoneservice        0.000229
gender              0.000117
dtype: float64

## 3.7 Correlación

In [147]:
df_full_train.tenure.max()

np.int64(72)

In [148]:
df_full_train[numerical].corrwith(df_full_train.churn) #correlación entre las 3 variables númericas y el churn

tenure           -0.351885
monthlycharges    0.196805
totalcharges     -0.196353
dtype: float64

#### Correlación entre tenure y churn

In [149]:
df_full_train[df_full_train.tenure <= 2].churn.mean() #porcentaje de personas que se van cuyo número de meses con la compañia es menor ogial a 2

np.float64(0.5953420669577875)

In [150]:
df_full_train[(df_full_train.tenure > 2) & (df_full_train.tenure <= 12)].churn.mean() #entre 2 y 12 meses

np.float64(0.3994413407821229)

#### Correlación entre monthly charges y churn

In [151]:
df_full_train[df_full_train.monthlycharges <= 20].churn.mean()

np.float64(0.08795411089866156)

In [152]:
df_full_train[(df_full_train.monthlycharges > 20) & (df_full_train.monthlycharges <= 50)].churn.mean() 

np.float64(0.18340943683409436)

In [153]:
df_full_train[(df_full_train.monthlycharges > 50) & (df_full_train.monthlycharges <= 100)].churn.mean() 

np.float64(0.33235581622678395)

## 3.8 One-hot encoding

#### Consiste en reescribir las variables categóricas de forma binaria

In [154]:
from sklearn.feature_extraction import DictVectorizer

In [155]:
train_dicts = df_train[categorical + numerical].to_dict(orient='records') #orient lo ordena mejor
train_dicts[0]

{'gender': 'female',
 'seniorcitizen': 0,
 'partner': 'yes',
 'dependents': 'yes',
 'phoneservice': 'yes',
 'multiplelines': 'yes',
 'internetservice': 'fiber_optic',
 'onlinesecurity': 'yes',
 'onlinebackup': 'yes',
 'deviceprotection': 'yes',
 'techsupport': 'yes',
 'streamingtv': 'yes',
 'streamingmovies': 'yes',
 'contract': 'two_year',
 'paperlessbilling': 'yes',
 'paymentmethod': 'electronic_check',
 'tenure': 72,
 'monthlycharges': 115.5,
 'totalcharges': 8425.15}

In [156]:
dv = DictVectorizer(sparse=False)

In [157]:
dv.fit(train_dicts) #se le enseña la forma en la que se van a entregar los datos

DictVectorizer(sparse=False)

In [158]:
X_train = dv.transform(train_dicts)
X_train.shape

(4225, 45)

#### Hacemos lo mismo pero con el set de validación

In [159]:
val_dicts = df_val[categorical + numerical].to_dict(orient='records') 

In [160]:
X_val = dv.transform(val_dicts)
X_val.shape

(1409, 45)

## 3.9 Regresión logistica

#### Regresa valores entre 0 y 1, como una probabilidad, la función se llama sigmoid

In [161]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [162]:
#z de prueba
z = np.linspace(-5, 5, 51)
z

array([-5. , -4.8, -4.6, -4.4, -4.2, -4. , -3.8, -3.6, -3.4, -3.2, -3. ,
       -2.8, -2.6, -2.4, -2.2, -2. , -1.8, -1.6, -1.4, -1.2, -1. , -0.8,
       -0.6, -0.4, -0.2,  0. ,  0.2,  0.4,  0.6,  0.8,  1. ,  1.2,  1.4,
        1.6,  1.8,  2. ,  2.2,  2.4,  2.6,  2.8,  3. ,  3.2,  3.4,  3.6,
        3.8,  4. ,  4.2,  4.4,  4.6,  4.8,  5. ])

In [163]:
sigmoid(z) #retorna valores entre 0 y 1

array([0.00669285, 0.00816257, 0.0099518 , 0.01212843, 0.01477403,
       0.01798621, 0.02188127, 0.02659699, 0.03229546, 0.03916572,
       0.04742587, 0.05732418, 0.06913842, 0.0831727 , 0.09975049,
       0.11920292, 0.14185106, 0.16798161, 0.19781611, 0.23147522,
       0.26894142, 0.31002552, 0.35434369, 0.40131234, 0.450166  ,
       0.5       , 0.549834  , 0.59868766, 0.64565631, 0.68997448,
       0.73105858, 0.76852478, 0.80218389, 0.83201839, 0.85814894,
       0.88079708, 0.90024951, 0.9168273 , 0.93086158, 0.94267582,
       0.95257413, 0.96083428, 0.96770454, 0.97340301, 0.97811873,
       0.98201379, 0.98522597, 0.98787157, 0.9900482 , 0.99183743,
       0.99330715])

In [164]:
def logistic_regression(xi):
    score = w0

    for j in range(len(w)):
        score = score + xi[j] * w[j]

    result = sigmoid(score)
    return result

## 3.10 Scikit-learn regresión logística

In [165]:
from sklearn.linear_model import LogisticRegression

In [166]:
model = LogisticRegression()
y_train.shape

(4225,)

In [167]:
model.fit(X_train, y_train) #ajustamos el módelo para hallar los pesos

/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [168]:
model.intercept_[0]

np.float64(-0.1109287445794238)

In [192]:
model.coef_[0].round(3) #pesos para hallar el score y luego la probalidad

array([ 0.564, -0.154, -0.519, -0.013, -0.095,  0.071, -0.098, -0.082,
       -0.033, -0.076, -0.354,  0.343, -0.098,  0.002, -0.251,  0.162,
       -0.02 ,  0.066, -0.098, -0.076,  0.251, -0.098, -0.261, -0.228,
        0.119, -0.121,  0.012, -0.095, -0.03 ,  0.045, -0.028,  0.162,
       -0.271,  0.221, -0.097, -0.098,  0.087, -0.05 , -0.098,  0.039,
        0.194, -0.098, -0.205, -0.072,  0.   ])

In [170]:
y_pred = model.predict_proba(X_train)[:, 1] #predict retorna los valores 0 o 1, predict_proba retorna dos probabilidades, la primera de que sea 0
#la otra de que sea 1, con [:, 1] mostramos solo la probabilidad de que sea 1

In [174]:
y_pred_val = model.predict_proba(X_val)[:, 1]

In [183]:
churn_decision = (y_pred_val >= 0.5) #lista binaria clientes que es probable que churneen
churn_decision

array([False, False, False, ..., False,  True,  True], shape=(1409,))

In [178]:
df_val[churn_decision].customerid #clientes que es probable que churneen

3       8433-wxgna
8       3440-jpscl
11      2637-fkfsy
12      7228-omtpn
19      6711-fldfb
           ...    
1397    5976-jcjrh
1398    2034-cgrhz
1399    5276-kqwhg
1407    6521-yytyi
1408    3049-solay
Name: customerid, Length: 322, dtype: object

In [180]:
y_val

array([0, 0, 0, ..., 0, 1, 1], shape=(1409,))

In [184]:
churn_decision.astype(int)

array([0, 0, 0, ..., 0, 1, 1], shape=(1409,))

In [186]:
(y_val == churn_decision).mean() #porcentaje que concuerdan

np.float64(0.801277501774308)

In [189]:
df_pred = pd.DataFrame()
df_pred['probability'] = y_pred_val #probabilidad de que se vayan
df_pred['prediction'] = churn_decision #gente que se va a ir (umbral del 50%)
df_pred['actual'] = y_val #valores reales

In [190]:
df_pred['correct'] = df_pred.prediction == df_pred.actual

In [191]:
df_pred

,probability,prediction,actual,correct
0,0.007863,False,0,True
1,0.209928,False,0,True
2,0.223710,False,0,True
3,0.561123,True,1,True
4,0.219586,False,0,True
...,...,...,...,...
1404,0.297790,False,0,True
1405,0.035463,False,1,False
1406,0.150149,False,0,True
1407,0.776681,True,1,True


## 3.11 Interpretación del modelo

In [193]:
dv.get_feature_names_out()

array(['contract=month-to-month', 'contract=one_year',
       'contract=two_year', 'dependents=no', 'dependents=yes',
       'deviceprotection=no', 'deviceprotection=no_internet_service',
       'deviceprotection=yes', 'gender=female', 'gender=male',
       'internetservice=dsl', 'internetservice=fiber_optic',
       'internetservice=no', 'monthlycharges', 'multiplelines=no',
       'multiplelines=no_phone_service', 'multiplelines=yes',
       'onlinebackup=no', 'onlinebackup=no_internet_service',
       'onlinebackup=yes', 'onlinesecurity=no',
       'onlinesecurity=no_internet_service', 'onlinesecurity=yes',
       'paperlessbilling=no', 'paperlessbilling=yes', 'partner=no',
       'partner=yes', 'paymentmethod=bank_transfer_(automatic)',
       'paymentmethod=credit_card_(automatic)',
       'paymentmethod=electronic_check', 'paymentmethod=mailed_check',
       'phoneservice=no', 'phoneservice=yes', 'seniorcitizen',
       'streamingmovies=no', 'streamingmovies=no_internet_service',

In [196]:
model.coef_[0].round(3)

array([ 0.564, -0.154, -0.519, -0.013, -0.095,  0.071, -0.098, -0.082,
       -0.033, -0.076, -0.354,  0.343, -0.098,  0.002, -0.251,  0.162,
       -0.02 ,  0.066, -0.098, -0.076,  0.251, -0.098, -0.261, -0.228,
        0.119, -0.121,  0.012, -0.095, -0.03 ,  0.045, -0.028,  0.162,
       -0.271,  0.221, -0.097, -0.098,  0.087, -0.05 , -0.098,  0.039,
        0.194, -0.098, -0.205, -0.072,  0.   ])

In [200]:
list(zip(dv.get_feature_names_out(), model.coef_[0].round(3))).sort() #une los pesos con los nombres de las características

#### Modelo pequeño con solo 3 variables

In [201]:
small = ['contract','tenure','monthlycharges']

In [226]:
df_train[small].iloc[:5].to_dict(orient='records') #para fittear al modelo

[{'contract': 'two_year', 'tenure': 72, 'monthlycharges': 115.5},
 {'contract': 'month-to-month', 'tenure': 10, 'monthlycharges': 95.25},
 {'contract': 'month-to-month', 'tenure': 5, 'monthlycharges': 75.55},
 {'contract': 'month-to-month', 'tenure': 5, 'monthlycharges': 80.85},
 {'contract': 'two_year', 'tenure': 18, 'monthlycharges': 20.1}]

In [227]:
dicts_train_small = df_train[small].to_dict(orient='records') #para fittear al modelo
dicts_val_small = df_val[small].to_dict(orient='records') #para fittear al modelo

In [211]:
dv_small = DictVectorizer(sparse = False) #creamos el modelo
dv_small.fit(dicts_train_small) #fitteamos el modelo para que acepte diccionarios de la forma anterior

DictVectorizer(sparse=False)

In [212]:
dv_small.get_feature_names_out() #revisamos que usamos las variables que queremos

array(['contract=month-to-month', 'contract=one_year',
       'contract=two_year', 'monthlycharges', 'tenure'], dtype=object)

In [213]:
X_train_small = dv_small.transform(dicts_train_small) #obtenemos la matriz de características para convertir variables categóricas en 0 y 1
X_train_small

array([[  0.  ,   0.  ,   1.  , 115.5 ,  72.  ],
       [  1.  ,   0.  ,   0.  ,  95.25,  10.  ],
       [  1.  ,   0.  ,   0.  ,  75.55,   5.  ],
       ...,
       [  1.  ,   0.  ,   0.  ,  90.  ,   2.  ],
       [  0.  ,   0.  ,   1.  ,  24.5 ,  27.  ],
       [  1.  ,   0.  ,   0.  ,  80.85,   9.  ]], shape=(4225, 5))

In [214]:
model_small = LogisticRegression()
model_small.fit(X_train_small, y_train) #hallamos los pesos

LogisticRegression()

In [228]:
w0 = model_small.intercept_[0] #intercepto
w0

np.float64(-2.477957595829565)

In [223]:
w = model_small.coef_[0] #pesos
w.round(3) 

array([ 0.971, -0.024, -0.948,  0.027, -0.036])

In [224]:
dict(zip(dv_small.get_feature_names_out(), w.round(3)))

{'contract=month-to-month': np.float64(0.971),
 'contract=one_year': np.float64(-0.024),
 'contract=two_year': np.float64(-0.948),
 'monthlycharges': np.float64(0.027),
 'tenure': np.float64(-0.036)}

## 3.12 Usando el modelo

In [233]:
dicts_full_train = df_full_train[categorical + numerical].to_dict(orient = 'records') #convertimos en diccionario

In [232]:
dv = DictVectorizer(sparse=False)

In [240]:
X_full_train = dv.fit_transform(dicts_full_train) #fitteamos el dict vectorizer y hallamos la matriz de caracteristicas

In [237]:
y_full_train = df_full_train.churn.values

In [241]:
model = LogisticRegression()
model.fit(X_full_train, y_full_train) #fitteamos el modelo

/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [242]:
dicts_test = df_test[categorical+numerical].to_dict(orient='records')

In [243]:
X_test = dv.transform(dicts_test) #hallamos la matriz de caracterisitas de test

In [247]:
y_pred = model.predict_proba(X_test)[:, 1] #columna de la probabilidad de que se vayan
y_pred

array([0.05704183, 0.12942998, 0.32184304, ..., 0.00652234, 0.19165844,
       0.66134701], shape=(1409,))

In [248]:
churn_decision = (y_pred >= 0.5)

In [249]:
(churn_decision == y_test).mean() 

np.float64(0.8140525195173882)

#### 80% de predicciones correctas